<a href="https://colab.research.google.com/github/HASTAR-CPP/Aionic/blob/main/Aionic-model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import pandas as pd
import numpy as np
from scipy.signal import butter, filtfilt
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from imblearn.combine import SMOTETomek
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import joblib

In [ ]:
# **Step 1: Load Dataset from Multiple CSV Files**
train_path = "/content/drive/MyDrive/TrainCSV_C23"
test_path = "/content/drive/MyDrive/TestCSV_C23"

def load_csv_from_folder(folder_path):
    all_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith(".csv")]
    df_list = [pd.read_csv(f) for f in all_files]
    return pd.concat(df_list, ignore_index=True) if df_list else None

In [ ]:
# Load train and test data
train_df = load_csv_from_folder(train_path)
test_df = load_csv_from_folder(test_path)

print("Training dataset shape:", train_df.shape)
print("Testing dataset shape:", test_df.shape)


In [ ]:
# **Step 2: Remove Unnecessary Columns**
def preprocess_dataframe(df):
    df.drop(columns=["FCR", "ECR"], errors="ignore", inplace=True)  # Remove wrist sensor data
    selected_sensors = ["FDS", "ED", "FDI", "Movement_Label"]
    df = df[selected_sensors]

    finger_movements = ["Finger Flexion", "Finger Extension", "Pinch Grip", "Open Hand", "Close Hand"]
    df = df[df["Movement_Label"].isin(finger_movements)]
    df.dropna(inplace=True)

    # Remove outliers
    for sensor in ["FDS", "ED", "FDI"]:
        df = df[(df[sensor] > df[sensor].quantile(0.01)) & (df[sensor] < df[sensor].quantile(0.99))]

    return df


In [ ]:
train_df = preprocess_dataframe(train_df)
test_df = preprocess_dataframe(test_df)

print("Train dataset after preprocessing:", train_df.shape)
print("Test dataset after preprocessing:", test_df.shape)


In [ ]:

# **Step 3: Apply Low-Pass Filter**
def butter_lowpass_filter(data, cutoff=100, fs=1000, order=4):
    nyquist = 0.5 * fs
    normal_cutoff = cutoff / nyquist
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return filtfilt(b, a, data)

for sensor in ["FDS", "ED", "FDI"]:
    train_df[sensor] = butter_lowpass_filter(train_df[sensor])
    test_df[sensor] = butter_lowpass_filter(test_df[sensor])


In [ ]:

# **Step 4: Feature Extraction (Sliding Window)**
window_size = 200
stride = 100

def extract_features_from_window(window_df):
    features = {}
    for sensor in ["FDS", "ED", "FDI"]:
        data = window_df[sensor]
        features[f"{sensor}_mean"] = np.mean(data)
        features[f"{sensor}_rms"] = np.sqrt(np.mean(data**2))
        features[f"{sensor}_var"] = np.var(data)
        features[f"{sensor}_mav"] = np.mean(np.abs(data))
        features[f"{sensor}_ssc"] = np.sum(np.diff(data) ** 2)
        features[f"{sensor}_zc"] = np.count_nonzero(np.diff(np.sign(data)))
        features[f"{sensor}_wl"] = np.sum(np.abs(np.diff(data)))

    features["Movement_Label"] = window_df["Movement_Label"].iloc[0]
    return features

def create_feature_dataset(df):
    processed_data = []
    for i in range(0, len(df) - window_size, stride):
        window_df = df.iloc[i:i + window_size]
        processed_data.append(extract_features_from_window(window_df))
    return pd.DataFrame(processed_data)

train_features = create_feature_dataset(train_df)
test_features = create_feature_dataset(test_df)


In [ ]:

# **Step 5: Normalize Features**
scaler = StandardScaler()
feature_columns = train_features.columns[:-1]
train_features[feature_columns] = scaler.fit_transform(train_features[feature_columns])
test_features[feature_columns] = scaler.transform(test_features[feature_columns])
joblib.dump(scaler, "scaler.pkl")


In [ ]:

# **Step 6: Balance the Dataset using SMOTETomek**
X_train = train_features[feature_columns].values
y_train = train_features["Movement_Label"].values.reshape(-1, 1)
X_test = test_features[feature_columns].values
y_test = test_features["Movement_Label"].values.reshape(-1, 1)

encoder = OneHotEncoder(sparse=False)
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)
joblib.dump(encoder, "encoder.pkl")

smote = SMOTETomek(random_state=42)
X_train, y_train = smote.fit_resample(X_train, y_train)


In [ ]:

# **Step 7: Save Processed Data**
np.savez("train_test_data_final.npz", X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test)


In [ ]:

# **Step 8: Train the ML Model**
model = keras.Sequential([
    keras.layers.Dense(256, activation="relu", input_shape=(X_train.shape[1],)),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(128, activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(64, activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(32, activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(5, activation="softmax")
])

model.compile(optimizer="adamw", loss="categorical_crossentropy", metrics=["accuracy"])

early_stopping = keras.callbacks.EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)

history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test), callbacks=[early_stopping])


In [ ]:

# **Step 9: Evaluate Model**
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"✅ Model Test Accuracy: {test_acc:.4f}")

y_pred = np.argmax(model.predict(X_test), axis=1)
y_test_labels = np.argmax(y_test, axis=1)
print("\nClassification Report:\n", classification_report(y_test_labels, y_pred))

plt.figure(figsize=(8,6))
cm = confusion_matrix(y_test_labels, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=encoder.categories_[0], yticklabels=encoder.categories_[0])
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.show()


In [ ]:
# **Step 10: Save the Model**
model.save("trained_emg_model.h5")
print("✅ Model saved as 'trained_emg_model.h5'")
